In [1]:
pip install numpy plotly ipython ipywidgets vtk

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Importing the required libraries
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import vtk
from vtk.util import numpy_support

# Load the data
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("mixture.vti")
reader.Update()

# Extract the data using the vtk_to_numpy utility
data = reader.GetOutput()
dims = data.GetDimensions()
scalars = data.GetPointData().GetScalars()
numpy_data = numpy_support.vtk_to_numpy(scalars).reshape(dims[2], dims[1], dims[0])
numpy_data = numpy_data.transpose(2, 1, 0)

# Flatten the data for plotting
values = numpy_data.flatten()
x, y, z = np.indices(numpy_data.shape)

# Initialize the plotly figures
iso_fig = go.FigureWidget(data=go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=values, isomin=0, isomax=0, surface_count=1,
    colorscale="plasma", cmin=values.min(), cmax=values.max(),
    showscale=False
))
hist_fig = go.FigureWidget(go.Histogram(
    x=values, nbinsx=30, marker_color='rgb(101, 109, 242)'
))

# Update the layout of the figures
iso_fig.update_layout(
    title='3D Isosurface Visualization',
    autosize=True,
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False),  
    )
)

hist_fig.update_layout(
    title='Scalar Value Distribution',
    autosize=True,
    xaxis_title='Vortex scalar values',
    yaxis_title='Frequency',
    bargap=0.01
)

# Define a slider for adjusting the isovalue
slider = widgets.FloatSlider(
    value=0.0, min=values.min(), max=values.max(), step=0.01,
    description='Isovalue:', readout_format='.2f', continuous_update=False
)

# Callback function for updating the plots
def update_plots(change):
    isovalue = change.new
    iso_fig.data[0].update(isomin=isovalue, isomax=isovalue)
    if isovalue == 0.0:
        filtered_values = values
        hist_fig.update_layout(xaxis=dict(range=[values.min(), values.max()]))
    else:
        filtered_values = values[(values >= isovalue - 0.25) & (values <= isovalue + 0.25)]
        hist_fig.update_layout(xaxis=dict(range=[isovalue - 0.25, isovalue + 0.25]))
    hist_fig.data[0].x = filtered_values

slider.observe(update_plots, 'value')

# Reset button
reset_button = widgets.Button(description='Reset', icon='undo', style={'button_color': 'skyblue'})

# Reset function
def on_reset_button_clicked(b):
    slider.value = 0.0

reset_button.on_click(on_reset_button_clicked)

# Layout
widgets_layout = widgets.Layout(display='flex', flex_flow='row', align_items='center', width='100%', height='800px')
hbox1 = widgets.HBox([slider, reset_button])
hbox2 = widgets.HBox([iso_fig, hist_fig])
main_box = widgets.VBox([hbox1, hbox2])

# Display the interface
display(main_box)